In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
sales = pd.read_csv("GrossSales.csv")

spellingMapping = {
    "clearlake": "clear lake",
    "dewitt": "de witt",
    "guttenburg" : "guttenberg",
    "grand mounds": "grand mound",
    "jewell": "jewell junction",
    "kellog": "kellogg",
    "ottuwma": "ottumwa",
    "otumwa": "ottumwa",
    "lemars": "le mars",
    "leclaire": "le claire"
}   

sales['City'] = sales['City'].map(lambda x: x.lower().replace("'", "").strip())

sales['City'] = sales['City'].map(lambda x: spellingMapping.get(x, x))
sales['City'] = sales['City'].map(lambda x: x.replace('mt', 'mount'))
sales['City'] = sales['City'].map(lambda x: x.replace('saint', 'st.'))
sales['City'] = sales['City'].map(lambda x: re.sub('^ft.', 'fort', x))
sales['City'] = sales['City'].map(lambda x: re.sub('^st ', 'st. ', x))

In [3]:
census = pd.read_csv("CensusData.csv")
census['City'] = census['City'].map(lambda x: x.lower().replace("'", "").strip())

In [4]:
censusunique = np.unique(census['City'])
for city in np.unique(sales['City']):
    if city not in censusunique:
        print(city)

colorado springs


Colorado Springs isn't a city in Iowa. We only lose 1 row by keeping Colorado Springs out, so we'll leave it out. 

In [5]:
len(sales.index)

121045

In [6]:
merged = sales.merge(census, on='City', how='inner')
merged = merged.drop("Unnamed: 0", axis = 1)

In [7]:
merged['Volume Per Capita'] = merged['Volume Sold (Liters)'] / merged['Population']
merged['Bottles Per Capita'] = merged['Bottles Sold'] / merged['Population']
merged['Sales Per Capita'] = merged['Sale (Dollars)'] / merged['Population']

In [8]:
len(merged.columns)

13

In [9]:
merged.head()

,City,Date,Volume Sold (Liters),Bottles Sold,Sale (Dollars),Population,HousingUnits,AreaSQMT,DensitySQMP,DensitySQMH,Volume Per Capita,Bottles Per Capita,Sales Per Capita
0,ackley,01/04/2016,114.00,118,1858.58,1589,789,2.48,648.6,322.0,0.071743,0.074261,1.169654
1,ackley,01/05/2015,163.99,158,1953.04,1589,789,2.48,648.6,322.0,0.103203,0.099434,1.229100
2,ackley,01/06/2014,146.50,126,1260.57,1589,789,2.48,648.6,322.0,0.092196,0.079295,0.793310
3,ackley,01/07/2013,98.75,96,955.93,1589,789,2.48,648.6,322.0,0.062146,0.060415,0.601592
4,ackley,01/08/2013,61.50,72,686.59,1589,789,2.48,648.6,322.0,0.038704,0.045312,0.432089


In [10]:
len(merged.City.unique())

418

In [11]:
one_hot_city = pd.get_dummies(merged.City)
merged = pd.merge(merged, one_hot_city, left_index=True, right_index=True)


merged.Date = pd.to_datetime(merged.Date, infer_datetime_format=True)
# The day of the week with Monday=0, Sunday=6
merged['Dayofweek'] = merged.Date.dt.dayofweek
# The week ordinal of the year
merged['Week'] = merged.Date.dt.weekofyear
merged['Month'] = merged.Date.dt.month

In [13]:
merged.head()

,City,Date,Volume Sold (Liters),Bottles Sold,Sale (Dollars),Population,HousingUnits,AreaSQMT,DensitySQMP,DensitySQMH,...,windsor heights,winfield,winterset,winthrop,woodbine,woodward,zwingle,Dayofweek,Week,Month
0,ackley,2016-01-04,114.00,118,1858.58,1589,789,2.48,648.6,322.0,...,0,0,0,0,0,0,0,0,1,1
1,ackley,2015-01-05,163.99,158,1953.04,1589,789,2.48,648.6,322.0,...,0,0,0,0,0,0,0,0,2,1
2,ackley,2014-01-06,146.50,126,1260.57,1589,789,2.48,648.6,322.0,...,0,0,0,0,0,0,0,0,2,1
3,ackley,2013-01-07,98.75,96,955.93,1589,789,2.48,648.6,322.0,...,0,0,0,0,0,0,0,0,2,1
4,ackley,2013-01-08,61.50,72,686.59,1589,789,2.48,648.6,322.0,...,0,0,0,0,0,0,0,1,2,1
